In [ ]:
# What works in this>>1) Tracing
# Install required packages
!pip install openai-agents
!pip install python-dotenv
!pip install gradio --upgrade
!pip install arize-phoenix
!pip install arize-phoenix-otel
!pip install openinference-instrumentation-openai-agents
!pip install openai --upgrade

import os
from openai import OpenAI
import nest_asyncio
import uuid
import asyncio
from pydantic import BaseModel
import gradio as gr
import tempfile

nest_asyncio.apply()

# Phoenix tracing setup
PHOENIX_API_KEY = "your_phoenix_api_key_here"
os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={PHOENIX_API_KEY}"
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com"

from phoenix.otel import register
tracer_provider = register(project_name="openai_agents", auto_instrument=True)

# OpenAI API setup
api_key = "your_openai_api_key_here"
os.environ['OPENAI_API_KEY'] = api_key
client = OpenAI(api_key=api_key)

from agents import (
    Agent, HandoffOutputItem, ItemHelpers, MessageOutputItem,
    RunContextWrapper, Runner, ToolCallItem, ToolCallOutputItem,
    TResponseInputItem, function_tool, handoff, trace
)
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX
from agents.tracing import set_tracing_export_api_key

set_tracing_export_api_key(api_key)

# Context for Indian legal queries
class LegalContextIndia(BaseModel):
    query_type: str | None = None
    state: str | None = None

# Tools (Enhanced with Indian Law Citations)
@function_tool
async def family_info_india(context: RunContextWrapper[LegalContextIndia], issue: str) -> str:
    if "divorce" in issue.lower():
        return "Under the Hindu Marriage Act, 1955 (Section 13(1)), divorce can be filed for cruelty, adultery, or desertion (e.g., 2+ years). Proceedings can be private in family court."
    elif "domestic" in issue.lower() or "violence" in issue.lower():
        return "The Protection of Women from Domestic Violence Act, 2005 (Section 3) defines domestic violence; Section 18 allows protection orders. Contact a magistrate anonymously if needed."
    return "No specific info available—routing to another expert."

@function_tool
async def tenancy_info_india(context: RunContextWrapper[LegalContextIndia], issue: str) -> str:
    if "eviction" in issue.lower():
        return "Under state-specific Rent Control Acts (e.g., Delhi Rent Control Act, 1958, Section 14), eviction requires 1-6 months’ notice, typically for non-payment or misuse."
    elif "landlord" in issue.lower():
        return "Landlords can’t forcibly evict without notice (Transfer of Property Act, 1882, Section 106). File a civil suit or police complaint under IPC Section 441 for trespass if harassed."
    return "No specific info available—routing to another expert."

@function_tool
async def offences_info_india(context: RunContextWrapper[LegalContextIndia], issue: str) -> str:
    if "accused" in issue.lower() or "theft" in issue.lower():
        return "Theft under IPC Section 378 is taking movable property without consent, punishable under Section 379 with up to 7 years. If accused, an FIR (CrPC Section 154) initiates investigation."
    elif "harassment" in issue.lower():
        return "Harassment under IPC Section 354 (assault to outrage modesty) carries up to 3 years. Anonymous reporting is possible under CrPC Section 154."
    return "No specific info available—routing to another expert."

@function_tool
async def consumer_info_india(context: RunContextWrapper[LegalContextIndia], issue: str) -> str:
    if "fraud" in issue.lower() or "cheated" in issue.lower():
        return "The Consumer Protection Act, 2019 (Section 17) allows complaints to District Forum (up to ₹1 crore) for fraud or unfair trade (Section 2(47)). No advocate needed."
    elif "refund" in issue.lower():
        return "Defective goods entitle you to refund/replacement under CPA, 2019 (Section 2(10)). File under Section 35 if unresolved by seller."
    return "No specific info available—routing to another expert."

# Agents
router_agent = Agent[LegalContextIndia](
    name="Legal Router India",
    instructions=f"{RECOMMENDED_PROMPT_PREFIX} Based on the user’s input, either provide a helpful and final summary of applicable Indian laws or, if more specific expertise is required, hand off silently to the most suitable expert agent. Do not respond to the user after initiating a handoff. Let the new agent provide the final response. Avoid asking any user questions. This is for general awareness—not legal advice.",
    handoffs=[]
)

civil_family_agent = Agent[LegalContextIndia](
    name="Civil Family Agent",
    instructions=f"{RECOMMENDED_PROMPT_PREFIX} Based on the user’s input, provide the most relevant and helpful summary of applicable Indian family laws. Avoid asking the user any questions or clarifications. Give a clear and final response based on statutes and known procedures, suitable for general awareness—not legal advice.",
    tools=[family_info_india]
)

civil_tenancy_agent = Agent[LegalContextIndia](
    name="Civil Tenancy Agent",
    instructions=f"{RECOMMENDED_PROMPT_PREFIX} Based on the user’s input, provide the most relevant and helpful summary of applicable Indian tenancy laws. Avoid asking the user any questions or clarifications. Give a clear and final response based on statutes and known procedures, suitable for general awareness—not legal advice.",
    tools=[tenancy_info_india]
)

criminal_offences_agent = Agent[LegalContextIndia](
    name="Criminal Offences Agent",
    instructions=f"{RECOMMENDED_PROMPT_PREFIX} Based on the user’s input, provide the most relevant and helpful summary of applicable Indian criminal laws or procedures. Avoid asking the user any questions or clarifications. Give a clear and final response based on statutes and known procedures, suitable for general awareness—not legal advice.",
    tools=[offences_info_india]
)

civil_consumer_agent = Agent[LegalContextIndia](
    name="Civil Consumer Agent",
    instructions=f"{RECOMMENDED_PROMPT_PREFIX} Based on the user’s input, provide the most relevant and helpful summary of applicable Indian consumer laws. Avoid asking the user any questions or clarifications. Give a clear and final response based on statutes and known procedures, suitable for general awareness—not legal advice.",
    tools=[consumer_info_india]
)

router_agent.handoffs.extend([civil_family_agent, civil_tenancy_agent, criminal_offences_agent, civil_consumer_agent])
civil_family_agent.handoffs.append(router_agent)
civil_tenancy_agent.handoffs.append(router_agent)
criminal_offences_agent.handoffs.append(router_agent)
civil_consumer_agent.handoffs.append(router_agent)

# Conversation State
class ConversationState:
    def __init__(self):
        self.current_agent = router_agent
        self.input_items = []
        self.context = LegalContextIndia()
        self.conversation_id = uuid.uuid4().hex[:16]

state = ConversationState()

# Audio Transcription (whisper-1, per API docs)
def transcribe_audio_file(audio_file_path):
    try:
        with open(audio_file_path, "rb") as audio_file:
            response = client.audio.transcriptions.create(
                model="gpt-4o-mini-transcribe",
                file=audio_file,
                response_format="text"
            )
            return response.strip() or "Sorry, couldn’t hear—try again!"
    except Exception as e:
        return f"Audio error: {e}"

# Text-to-Speech (tts-1)
def text_to_speech(text, language="en"):
    try:
        supported_languages = ["en", "hi", "ta", "bn", "te", "mr"]
        if language not in supported_languages:
            language = "en"
        response = client.audio.speech.create(
            model="tts-1",
            voice="alloy",
            input=text
        )
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
        response.stream_to_file(temp_file.name)
        return temp_file.name
    except Exception as e:
        return None

# Handler with Single Submit and Status
async def legal_advisor(message, audio, language):
    global state
    input_text = ""
    if audio:
        input_text = transcribe_audio_file(audio)
    elif message:
        input_text = message

    if not input_text:
        return "Please provide a question via text or audio.", None

    state.input_items.append({"content": input_text, "role": "user"})

    # Process until final response (handle handoffs in one go)
    response_text = ""
    while True:
        result = await Runner.run(state.current_agent, state.input_items, context=state.context)
        new_response = ""
        assigned_agent = state.current_agent.name

        for new_item in result.new_items:
            if isinstance(new_item, MessageOutputItem):
                new_response += f"{new_item.agent.name}: {ItemHelpers.text_message_output(new_item)}\n"
            elif isinstance(new_item, HandoffOutputItem):
                state.current_agent = new_item.agent
                assigned_agent = new_item.agent.name

        if new_response and not any(isinstance(item, HandoffOutputItem) for item in result.new_items):
            # Final response reached
            response_text = f"Assigned to {assigned_agent}\n\n" + new_response
            break
        elif not new_response:
            response_text = "No specific info found—try rephrasing your question."
            break

        state.input_items = result.to_input_list()

    response_text = f"Note: This is general info under Indian law, not legal advice. Consult an advocate for your case. Your query remains anonymous.\n\n{response_text}"
    response_audio = text_to_speech(response_text, language)

    return response_text, response_audio if response_audio else "Audio generation failed."

# Gradio Interface
demo = gr.Interface(
    fn=legal_advisor,
    inputs=[
        gr.Textbox(label="Type Your Question (Optional)", placeholder="Or upload audio below..."),
        gr.Audio(sources=["upload"], type="filepath", label="Upload Anonymous Audio Question"),
        gr.Dropdown(
            choices=["en (English)", "hi (Hindi)", "ta (Tamil)", "bn (Bengali)", "te (Telugu)", "mr (Marathi)"],
            value="en (English)",
            label="Select Response Language (Audio)"
        )
    ],
    outputs=[
        gr.Textbox(label="Response Text"),
        gr.Audio(label="Response Audio")
    ],
    title="Legal Opinion Request (India)",
    description="Submit anonymous audio or text for general info on sensitive legal issues under Indian law—not advice! Get responses in text and audio (choose language).",
    examples=[["Can I divorce quietly?"], ["My landlord is harassing me"], ["I’m accused of theft—what now?"], ["A shop cheated me—help!"]]
)
demo.queue()  # Enables async + queue support for multiple sessions
demo.launch(share=True, debug=True)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: openai_agents
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://app.phoenix.arize.com/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {'api_key': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://dc0e55fb17af94666b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co

<ipython-input-5-ad8c890500a2>:154: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(temp_file.name)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://dc0e55fb17af94666b.gradio.live
